In [71]:
import torch
import numpy as np
import os
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch.optim as optim 
import torch.nn as nn

In [72]:

learning_rate = 0.003
N_Epochs      = 20
batch_size    = 32



### Get the data

In [73]:
data_path = "data/MNISTdata/"
mnist_train = datasets.MNIST(data_path, train = True , download = True)



In [74]:
mnist_test = datasets.MNIST(data_path, train = False, download = True)

In [75]:
mnist_train_tr = datasets.MNIST(data_path , train = True, download = False,
                               transform = transforms.Compose([transforms.ToTensor()]))

mnist_test_tr = datasets.MNIST(data_path , train = False, download = False,
                               transform = transforms.Compose([transforms.ToTensor()]))

### Display Image

In [76]:
Image.fromarray(mnist_test_tr.data[123].numpy())

In [77]:
Image.fromarray(mnist_test_tr.data[12].numpy())

In [79]:
Image.fromarray(mnist_test_tr.data[1].numpy())

In [80]:
### mnist_train_tr = mnist_train_tr.data (60000, 1, 28, 28)

mnist_train_tr.data.shape

torch.Size([60000, 28, 28])

In [81]:
mnist_test_tr.data.shape

torch.Size([10000, 28, 28])

### Create Data Loaders

In [95]:
batch_size = 32

train_dl  = torch.utils.data.DataLoader(mnist_train_tr, batch_size=batch_size, shuffle=True  ) 

test_dl   = torch.utils.data.DataLoader(mnist_test_tr,  batch_size=10000,      shuffle=False ) 


### Metrics function

In [116]:

def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure

### CNN Architecture

In [117]:

class Classifier_CNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()
            
        self.model = nn.Sequential(
                
                ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2), 
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
 
                ## feed forward layer 
                nn.Flatten(),
                nn.Linear(800, 1024),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(1024, 10),
                nn.LogSoftmax(dim=1)
        )
   
    def forward(self, inputs):
            
        return self.model(inputs)
    
    


### Training the model

In [118]:
def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    losses_list = []
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out as [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
       
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list

In [119]:
### train the model

In [120]:

model          = Classifier_CNN()

opt            = torch.optim.Adam(    model.parameters(), lr=learning_rate )

loss_fn        = nn.CrossEntropyLoss( )   

my_losses_list = training_loop(  N_Epochs, model, loss_fn, opt  ) 

0 loss= tensor(0.1025, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.2741, grad_fn=<NllLossBackward0>)
2 loss= tensor(1.7581e-05, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0002, grad_fn=<NllLossBackward0>)
4 loss= tensor(0.0521, grad_fn=<NllLossBackward0>)
5 loss= tensor(0.0005, grad_fn=<NllLossBackward0>)
6 loss= tensor(5.3193e-06, grad_fn=<NllLossBackward0>)
7 loss= tensor(0.0003, grad_fn=<NllLossBackward0>)
8 loss= tensor(0.0040, grad_fn=<NllLossBackward0>)
9 loss= tensor(0.0220, grad_fn=<NllLossBackward0>)
10 loss= tensor(0.0023, grad_fn=<NllLossBackward0>)
11 loss= tensor(1.8526e-05, grad_fn=<NllLossBackward0>)
12 loss= tensor(3.7253e-08, grad_fn=<NllLossBackward0>)
13 loss= tensor(0.1074, grad_fn=<NllLossBackward0>)
14 loss= tensor(0.0643, grad_fn=<NllLossBackward0>)
15 loss= tensor(2.7939e-06, grad_fn=<NllLossBackward0>)
16 loss= tensor(7.8231e-08, grad_fn=<NllLossBackward0>)
17 loss= tensor(1.7509e-07, grad_fn=<NllLossBackward0>)
18 loss= tensor(6.9662e-07, grad_fn=<NllLossBa

### Predicting and Evaluate with out Trained  Model

In [121]:

with torch.no_grad():
    for x_real, y_real in test_dl:
        
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function(y_real, preds)

Accuracy: 0.99
Confusion Matrix:
[[ 973    0    1    0    0    0    3    1    0    2]
 [   1 1119    2    3    0    1    4    3    2    0]
 [   0    0 1026    0    0    0    0    3    3    0]
 [   0    0    2 1008    0    0    0    0    0    0]
 [   0    0    1    0  962    0    1    2    1   15]
 [   1    0    0   10    0  875    1    1    2    2]
 [   0    1    1    0    1    0  953    0    2    0]
 [   0    0    2    4    1    0    1 1013    3    4]
 [   0    0    3    1    0    0    0    0  967    3]
 [   0    0    2    6    4    2    0    1    1  993]]
Precision: 0.989
Recall: 0.989
F1-mesure: 0.989
